# Neutralization curves for 8.9F antibody single mutant validations

In [ ]:
# Imports
import os
import warnings
import neutcurve
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import display

# Plotting colors
tol_muted_adjusted = [
    "#000000",
    "#CC6677", 
    "#1f78b4", 
    "#DDCC77", 
    "#117733", 
    "#882255", 
    "#88CCEE",
    "#44AA99", 
    "#999933", 
    "#AA4499", 
    "#EE7733",
    "#CC3311",
    "#DDDDDD",
]

# seaborn style settings
sns.set(rc={"figure.dpi":300, "savefig.dpi":300})
sns.set_style("ticks")
sns.set_palette(tol_muted_adjusted)

# Suppress warnings
warnings.simplefilter("ignore")

In [ ]:
# Read data
df = pd.read_excel("data/081023_fraction_infected.xlsx")
# Set output directory
out_dir = "figures/"

In [ ]:
# Rename viruses from plasmid number to mutation
rename_dict = {
    3017 : "Unmutated",
    3893 : "N89D",
    3894 : "N119S",
    3895 : "K125L",
    3896 : "K126L",
    3897 : "Y129L",
    3898 : "S135L",
    3899 : "S138N",
    3900 : "N148R",
    3901 : "Q149H",
    
}
df["virus"] = df["virus"].replace(rename_dict)

# Drop VSVG
df = df.loc[df["virus"] != "VSVG"]

df.head()

In [ ]:
# Fit hill curves using neutcurve
fits = neutcurve.curvefits.CurveFits(
    data=df,
    fixbottom=0,
    fixtop=1,
)

# IC values to calculate
fitParams = fits.fitParams(ics=[50, 80, 90, 95, 97, 98, 99])
# Show calculated values
display(fitParams)

In [ ]:
# Show calculated IC values
display(fitParams[fitParams["serum"]=="8.9F"][["serum","virus","ic50", "ic80", "ic90", "ic95", "ic97", "ic98", "ic99"]])

In [ ]:
# Markers
markers = [
    "o",
    "o",
    "o",
    "o",
    "o",
    "o",
    "o",
    "o",
    "o",
    "o",
    "o",
]

fig, axes = fits.plotSera(
    colors=tol_muted_adjusted,
    markers=markers,
    max_viruses_per_subplot=11,
    xlabel="",
    ylabel="",
    attempt_shared_legend=False,
)
axes[0,0].set_title(
    "8.9F", 
    weight="bold",
    fontsize=8,
)
axes[0,0].set_xlabel(
    "concentration (\u03BCg/mL)", 
    fontsize=8,
    # weight="bold",
)
axes[0,0].set_ylabel(
    "fraction infectivity", 
    fontsize=8,
    # weight="bold",
)
axes[0,0].set_ylim(-0.1, 1.3)
axes[0,0].set_yticks([0, 0.5, 1.0])
axes[0,0].set_yticklabels(labels=[0, 0.5, 1.0], fontsize=8)
axes[0,0].set_xlim(0.0005, 15)
axes[0,0].set_xticks([0.001, 0.01, 0.1, 1, 10])
axes[0,0].set_xticklabels(labels=["$10^{-3}$", "$10^{-2}$", "$10^{-1}$", "$10^0$", "$10^1$"], fontsize=8)
plt.setp(axes[0,0].collections, alpha=0.8, linewidths=0.5, colors="black") # for vertical error bar segment
plt.setp(axes[0,0].lines, alpha=0.8, markeredgewidth=0.5, markeredgecolor="black", linewidth=1) # for the lines and markers
sns.move_legend(
    axes[0,0], 
    bbox_to_anchor=(1.05, 1), 
    loc="upper left",
    borderaxespad=0,
    frameon=False,
    fontsize=8,
    title="amino acid\nsubstitutions",
    title_fontproperties={"weight" : "bold", "size" : 8},
    alignment="left"
)

# Add edges to legend markers to match scatter plot
for ha in axes[0,0].legend_.legendHandles:
    ha.set_markeredgecolor("black")
    ha.set_markeredgewidth(0.5)
    ha.set_linewidth(0)
    
# Change all spines
for axis in ["top", "bottom", "left", "right"]:
    axes[0,0].spines[axis].set_linewidth(1)
axes[0,0].tick_params(axis="both", length=4, width=1)

width = 3
height = 2
fig.set_size_inches(width, height)

# Make output dir if doesn't exist
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

plt.savefig(
    out_dir + "validation_neut_curves_89F.svg",
)